In [123]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import string
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup


In [124]:
tweet= pd.read_csv('train.csv')
test_tw=pd.read_csv('test.csv')
tweet.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [125]:
df=pd.concat([tweet,test_tw], sort="False")
df.shape

(10876, 5)

In [126]:
url = re.compile(r'https?://\S+|www\.\S+')

def get_URL(text):
    u = re.findall(url, text)
    return u if u else ''  

def remove_URL(text):
    return re.sub(url, r"", text)

def remove_HTML(text):
    soup = BeautifulSoup(text)
    return soup.get_text()

# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
def get_emoji(text):
    u = re.findall(emoji_pattern, text)
    return u

def remove_emoji(text):
    return emoji_pattern.sub(r'', text)

def get_tags(x):
    return {str(tag.strip("#")) for tag in x.split() if tag.startswith("#")}


In [127]:
df['text_original'] = df["text"]
df['text'] = df['text'].apply(lambda x: remove_HTML(x))

In [128]:
#предположим, что url пригодится, как отдельный признак. От url можно title раздобыть
df['url'] = df['text'].apply(lambda x: get_URL(x))
df['text'] = df['text'].apply(lambda x: remove_URL(x))

In [129]:
#предположим, что emoji пригодятся, как категориальный признак
df["emoji"] = df['text'].apply(lambda x: get_emoji(x))
df['text'] = df['text'].apply(lambda x: remove_emoji(x))

In [130]:
#вытаскиваем хештеги
df["tags"] =  df['text'].apply(lambda x: get_tags(x))

In [131]:
#убираем пунктуацию
df['text'] = df['text'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))


In [132]:
df[50:60]

,id,keyword,location,target,text,text_original,url,emoji,tags
50,73,ablaze,"Sheffield Township, Ohio",1.0,Deputies Man shot before Brighton home set abl...,Deputies: Man shot before Brighton home set ab...,[http://t.co/gWNRhMSO8k],[],{}
51,74,ablaze,India,1.0,Man wife get six years jail for setting ablaze...,Man wife get six years jail for setting ablaze...,[http://t.co/eV1ahOUCZA],[],{}
52,76,ablaze,Barbados,0.0,SANTA CRUZ ÛÓ Head of the St Elizabeth Police...,SANTA CRUZ ÛÓ Head of the St Elizabeth Police...,"[http://t.co/vplR5Hka2u, http://t.co/SxHW2TNNLf]",[],{}
53,77,ablaze,Anaheim,1.0,Police Arsonist Deliberately Set Black Church ...,Police: Arsonist Deliberately Set Black Church...,[http://t.co/pcXarbH9An],[],{}
54,78,ablaze,Abuja,0.0,Noches ElBestia AlexisSanchez happy to see my ...,Noches El-Bestia '@Alexis_Sanchez: happy to se...,[http://t.co/uc4j4jHvGR'],[],{}
55,79,ablaze,USA,1.0,Kurds trampling on Turkmen flag later set it a...,#Kurds trampling on Turkmen flag later set it ...,[http://t.co/4IzFdYC3cg],[],"{Kurds, Diyala}"
56,80,ablaze,South Africa,1.0,TRUCK ABLAZE R21 VOORTREKKER AVE OUTSIDE OR T...,TRUCK ABLAZE : R21. VOORTREKKER AVE. OUTSIDE O...,[http://t.co/8kscqKfKkF],[],{}
57,81,ablaze,"Sao Paulo, Brazil",0.0,Set our hearts ablaze and every city was a gif...,Set our hearts ablaze and every city was a gif...,[https://t.co/cYoMPZ1A0Z],[],{}
58,82,ablaze,hollywoodland,0.0,They sky was ablaze tonight in Los Angeles Im ...,They sky was ablaze tonight in Los Angeles. I'...,,[],{}
59,83,ablaze,"Edmonton, Alberta - Treaty 6",1.0,How the West was burned Thousands of wildfires...,How the West was burned: Thousands of wildfire...,"[http://t.co/iCSjGZ9tE1, http://t.co/9FxmN0l0Bd]",[],"{energy, climate, California}"


In [ ]:
from spellchecker import SpellChecker
spell = SpellChecker()
list = ['wildfire', 'wildfires', 'rockyfire', 'ebay', 'coworker', 'youtube', 'hailstorm', 'instagram', 'ebola', 'iphone']
for l in list:
    spell.word_frequency.add(l)

def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    if len(misspelled_words) > 0:
        print(misspelled_words)
        for word in misspelled_words:
            text = re.sub(word, spell.correction(word), text)
            #print(text)
    return text

#print("resp:", correct_spellings("corect my nam, ples"))
        
df['text']=df['text'].apply(lambda x : correct_spellings(x))


{'sask', 'ronge'}
{'cafire', 'hwy'}
{'fvck', 'haha'}
{'tampabay'}
{'bago'}
{'goooooooaaaaaal'}
{'looooool'}
{'wayi'}
{'nyc'}
{'cooool'}
{'bbcmtd'}
{'africanbaze', 'newsnigeria'}
{'phdsquares', 'theyve'}
{'inec', 'abia'}
{'\x89ûó', '\x89ûò', 'superintende'}
{'nsfw'}
{'youre'}
{'ancop'}
{'soooo', 'southridgelife'}
{'tracklist'}
{'nsfw'}
{'lanford', '\x89ûó'}
{'carolinaåêablaze'}
{'elbestia', 'alexissanchez', 'noches'}
{'diyala'}
{'intl', 'voortrekker'}
{'\x89û'}
{'dvd', 'gtxrwm', 'wmv'}
{'2k13', 'jacinta'}
{'navista7', 'news24680'}
{'nowplaying', 'edm'}
{'nxwestmidlands'}
{'cuz'}
{'8m', 'i24', 'nashvilletraffic'}
{'bayarea', 'us101', 'santaclara', 'pkwy'}
{'otleyhour', 'personalinjury'}
{'stlouis', 'tee\x89û', 'caraccidentlawyer'}
{'bigrigradio'}
{'i77', 'mooresville', 'iredell'}
{'sleepjunkies'}
{'hwymagellan', 'cabrillo'}
{'i77', 'mooresville', 'iredell'}
{'accidentwho'}
{'didnt'}
{'truckcrash', 'fortworth', 'youve'}
{'ashville'}
{'i540'}
{'cadfyi', 'rdhorndale', 'damagenhs999', 'piner

{'fantabulous', 'pfannebeckers'}
{'control\x89û\x9d', '\x89ûïhatchet', 'grabbers'}
{'udhampur'}
{'stans'}
{'isil'}
{'dattomm', '4head'}
{'liveleak'}
{'\x89ûª93', 'yeda', 'yakub', 'mumbai'}
{'4suspected', 'waziristan', 'acebreakingnews', 'etribune'}
{'udhampur', 'allthenews', 'livemint'}
{'obama'}
{'smb', 'darkreading'}
{'handside'}
{'albertas', 'cdnpoli', 'notleys', 'ableg'}
{'caixxum5sos'}
{'attackshare', 'contries', 'pakthey', 'shud'}
{'obamadhs'}
{'ary'}
{'control\x89û\x9d', '\x89ûïhatchet', 'grabbers'}
{'ignoranceshe', 'blazerfan', 'morebut', 'latinoand', 'gop', 'benothing'}
{'daytonarea'}
{'grabbers', '\x89ûïhatchet', 'nwo', 'infowars', 'control\x89û\x9d'}
{'pugprobs'}
{'udhampur'}
{'nickcocofree', 'jdabe80', 'envw98', 'juliedicaro'}
{'messeymetoo'}
{'didnt'}
{'transgendered'}
{'streamyxhomesouthern'}
{'loosers', 'ik'}
{'nickcocofree', 'jdabe80', 'envw98', 'juliedicaro'}
{'eunicenjoki', 'aiii', 'shes'}
{'satoshis', 'bitcoin', 'robotcoingame', 'robotlvl', 'freebitcoin'}
{'tweets', 

{'asf'}
{'blizzarddraco', 'lonewolffur'}
{'bubblycuteone', 'okayyyyyy', 'ima'}
{'lonewolffur'}
{'hearthstone'}
{'xbox', 'tcgreno'}
{'activision', 'cognizant'}
{'hearthstone'}
{'butterfinger'}
{'cuz'}
{'bruh', 'lonewolffur'}
{'chiefcg', 'haha'}
{'nailreal'}
{'boxset', 'buroker'}
{'bookanother'}
{'dialyses'}
{'xbox', 'setzorah', 'smh'}
{'hem712c'}
{'chamberedblood', 'speakingfromexperience'}
{'decisionsondecisions', '20s'}
{'imouto', 'isnt'}
{'bruh'}
{'dangerousbeans'}
{'scotto519'}
{'shii'}
{'aint'}
{'hastle'}
{'fuckin', 'sbee'}
{'loboparanoico'}
{'rockn', 'nightsad'}
{'ouvir', 'acesse', 'rolandonabeats', 'nosso'}
{'olrules'}
{'anthxvy'}
{'sethalphaeus', 'paramore', '5sos', 'favorites'}
{'cantar', 'pone'}
{'everwhe'}
{'blood\x89û'}
{'looooooooooool'}
{'piercings'}
{'aggressif'}
{'bbloggers', 'beram0s', 'butterlondon'}
{'resigninshame', 'emailed', 'argh', 'slsandpet'}
{'lauradeholanda'}
{'bdisgusting', 'tower\x89ûª', 'king\x89ûªs', '\x89û÷the', 'stephenking', 'thedarktower'}
{'smh'}
{'tr

{'minsuwoongs'}
{'recip'}
{'wheres'}
{'oooooohhhh', 'mmmmmm'}
{'louistomlinson'}
{'fewmoretweets'}
{'atleast', 'theyre'}
{'anhqdc', 'aiiamericangiri', 'foxnewsvideo'}
{'dhsscitech', 't\x89û', 'buildings\x89ûówe', 'forestservice'}
{'\x89û', 'duckvillelol', 'kca', 'votejkt48id'}
{'be4'}
{'themagickidraps'}
{'delanys', 'dhalgren'}
{'mofanon', 'retweet'}
{'booradleyvancullen'}
{'coquitlam'}
{'storen'}
{'cjoyner', 'minimehh', 'blacklivesmatter'}
{'phalaborwa'}
{'theyd'}
{'strikesstrikes'}
{'kshllcenterpri1', 'progress4ohio'}
{'theyre'}
{'nvr'}
{'potus'}
{'marcoarment'}
{'\x89ûïwe', 'jeb', 'mitt\x89û\x9d'}
{'omfg'}
{'slashandburn'}
{'jamaicaobserver', 'cnewslive'}
{'\x89ûïwe', 'jeb', 'mitt\x89û\x9d'}
{'alhanda', 'doricreates'}
{'appreciativeinquiry'}
{'standwithpp'}
{'dacherryontop13'}
{'scaryeven'}
{'jamaicaobserver', 'cnewslive'}
{'johnfromcranber'}
{'\x89ûïwe', 'jeb', 'mitt\x89û\x9d'}
{'wolter', 'la\x89û', 'smh', 'peeters'}
{'govts'}
{'\x89ûïwe', 'jeb', 'mitt\x89û\x9d'}
{'warfighting'}
{'

{'huebels'}
{'coxytown', 'spicybreads'}
{'mansehra', 'mi17', 'offr', 'muzzamil'}
{'peritoengrafoscopia'}
{'4am', 'didnt'}
{'neileastwood77', '\x89û', 'knobhead'}
{'brianroemmele', 'emv'}
{'mansehra', 'shaheed', 'hamayun', 'mi7'}
{'rosewell'}
{'weloveyoulouis', 'ipod', 'mtvhottest'}
{'johndcgow'}
{'olliebailey11', 'havnt'}
{'windows10'}
{'yahoofinancehope'}
{'mansehra'}
{'\x89ûò'}
{'countrys'}
{'yday', '2day'}
{'mansehra', 'mi17', 'offr', 'muzzamil'}
{'hicksville'}
{'mansehra'}
{'skype', 'smusx16475'}
{'theyre'}
{'truestory', 'modestmouseremix'}
{'mhtw4fnet', 'countrys', 'f496d'}
{'ttw'}
{'skh', 'ashton5sos', 'calum5sos', 'michael5sos', '5sos', 'luke5sos'}
{'mh370'}
{'youre', 'kuualohax'}
{'womem', 'mommyisbomb'}
{'jorrynja', 'terell', 'bfgfcrush'}
{'idk'}
{'wce', 'explicitpretty'}
{'aintsheperty', 'taykreidler', 'loveyouuuu', 'wedneday'}
{'yourboyshawn', 'bestfriends'}
{'buted'}
{'samelsamel'}
{'crushit', 'experiencesmh'}
{'crushmtvhottest', 'pydisney', 'bieber'}
{'master0fsloths'}
{'m

...

{'osp'}
{'kombat'}
{'hiroshimasized', 'hiroshima70'}
{'tch', 'whitbourne', 'rcmp'}
{'biterelated'}
{'fatalitiesxray'}
{'visionzero', 'motordom'}
{'segas', 'i\x89ûªve', 'kombat'}
{'uber'}
{'mkx', 'kombat', 'kevinedwardsjr', 'fatalitiesxrays'}
{'redlight'}
{'tch', 'whitbourne', 'rcmp', 'twovehicle'}
{'400000\x89ûò800000'}
{'8th', 'healthandsafety'}
{'nyc', 'cecanfs', 'policylab'}
{'freemarketeer', 'kurtschlichter', 'dibang'}
{'\x89û', 'capsizes', 'enca'}
{'overzero', 'questionfatalityflawless', 'lpunbiggiewrap', 'jaybig', 'ebrointheam'}
{'fatalityus', 'jakeadavis'}
{'trulystings'}
{'gangstermail'}
{'vgbootcamp', 'zss', 'vabengal', 'sws'}
{'crbzfz'}
{'jittering', 'hripsk'}
{'babality', 'jaycootchi', 'tellyfckngo', 'lmfaoooo', 'homie', 'sond'}
{'kosciusko'}
{'bardissimo'}
{'tonyakappes11'}
{'fatalityus'}
{'highestranking'}
{'babybackreeve'}
{'utica', 'aveblack'}
{'pxnatosil', 'renunciedilma'}
{'fatalityuudlk', 'jaxmk2'}
{'mileena'}
{'chrisman528'}
{'kosciusko'}
{'vgbootcamp', 'zss', 'vaben

{'kotaweather', 'sturgis'}
{'unr', 'wy', 'p\x89û'}
{'\x89û'}
{'darkndtatted'}
{'60mph', 'okwx'}
{'unr', 'wy', 's\x89û'}
{'postering', 'mothernature', 'calgaryfringe', 'hailstorm', 'yycfringe', 'killhard'}
{'ofclans', 'hailstorm', 'haleywhaley'}
{'autobody', 'peterhowenecn', 'hailstorm', 'necn'}
{'isnt'}
{'pokemon', 'hailthe', 'hailstorm'}
{'yyc', 'adriasimon', 'hailstorm', 'yycstorm', 'round2'}
{'crazyweather', 'yyc', 'hailstorm'}
{'hailstorm'}
{'hailstorm'}
{'hailstorm'}
{'bobcats'}
{'hellonwheelsamc', 'talkinghell', 'hailstorm', 'errrr', 'howfans', 'notgoingoutinthat'}
{'yyc', 'sto\x89û', 'abstorm', 'yycweather', 'hailstorm', 'captureyyc', 'haildamage'}
{'bobcats'}
{'hailstorm'}
{'xpost', '4608ìñ2474', '\x89û', 'hailstorm', 'rcityporn'}
{'20k', 'hailstorm'}
{'youre', 'lovemyjob', 'carlilescanoelivery'}
{'cowx', 'hailstorm'}
{'hailstorm', 'councilscc'}
{'jamaicaplain', 'hailstorm\x89û'}
{'yyc', 'iamdowntown', 'abstorm', 'hailstorm'}
{'citys'}
{'jamaicaplain', 'hailstorm'}
{'hail\x89û'

{'bluedio', 'blk', 're\x89û'}
{'meteoearth', 'lavapixcom'}
{'hurricanesurge', 'chubbysquirrel'}
{'shanaynay'}
{'shootas'}
{'lembra', '30stm'}
{'engle', 'byr'}
{'freegeezy17'}
{'nowplaying'}
{'brittsand9', 'loko', 'mixxtail'}
{'bluedio', 'blk', 're\x89û'}
{'meteoearth', 'vinustrip'}
{'hurricanetyphoon', 'ratingscategories'}
{'angelriveralib\x89û', 'h\x89û'}
{'hwrf'}
{'forgeting', 'fecal', 'foxnews', 'pattonoswalt', 'turdnado', 'scifi'}
{'wedaug5th'}
{'diarrhea'}
{'bluedio', 'blk', 're\x89û'}
{'websites'}
{'hurricanedolce'}
{'udhampur'}
{'jeffersondoris', 'sherfield72', 'updateme'}
{'meshnewsdesk', 'urogyn'}
{'yug'}
{'suryaray', 'injured\x89û', 'udhampur'}
{'yeat'}
{'trophyhunt', 'wakeupflorida', 'killedinjured'}
{'rvacchianonydn', 'arent'}
{'youre', 'chikislizeth08'}
{'war\x89û', 'leveled', 'google'}
{'udhampur'}
{'blevins'}
{'michaelgbaron', 'doesnt'}
{'udhampur'}
{'paulista', 'wilshere'}
{'leveled'}
{'ogun', 'shootoutåê'}
{'cosponsor', 'renew911health', 'christiec733'}
{'leveled'}
{'l

{'4playthursdays'}
{'asbury', 'asburyparkpress'}
{'i\x89ûªve', 'jmcwrites', 'pitchwars'}
{'hannemans'}
{'diamondkesawn', 'raynbowaffair', 'ramag'}
{'akcsl'}
{'itsjustinstuart'}
{'nosurrender'}
{'droppd', 'meelllttting', 'dubloadz'}
{'chopras', 'deepak'}
{'drdrewhln', 'areva'}
{'mattytalks', 'mwednesday', 'adultblackmale', 'tinybaby'}
{'kinkyconnors', 'lmao'}
{'instagram', 'demis'}
{'nashhmu'}
{'beccacaitlyn99'}
{'microsofts'}
{'dnbheaven', 'ep016'}
{'intelligencebar', 'aleisstokes', 'seagull07'}
{'g20', 'justintrudeau'}
{'tangletalk', 'joinvroom', 'crowdtappers'}
{'dmoneydemi', 'forsure', 'watchout'}
{'meltdowni', 'nprfreshair'}
{'commoditiesåêare'}
{'emini'}
{'youre', 'facebook', 'padres'}
{'pams'}
{'lemairelee', 'danharmon'}
{'barack', 'obama', 'examinercom'}
{'bullpen', 'warthen'}
{'milc5040h', 'paracord', 'warriorcord'}
{'furtrix', 'theyre', 'cougars'}
{'ushanka', 'xl6162'}
{'by\x89û'}
{'ymcglaun'}
{'univsfoundation'}
{'magner'}
{'ushanka'}
{'talibans', 'kiranahmedd'}
{'10th'}
{'m4

{'fpine', 'brandonmulcahy'}
{'kill\x89û'}
{'robbiewilliams', 'fkn', 'asswipe'}
{'police\x89û'}
{'crackin', 'arizonadot', 'salado'}
{'bridgeport'}
{'cus', 'blowin'}
{'wasn8217t'}
{'lastma'}
{'ticketed', 'helpme'}
{'socialmedia', 'facebook'}
{'slosheriff'}
{'quarantine\x89û', 'amageddon', 'reddit', 'onlinecommunities', 'freespeech'}
{'reddit'}
{'technews', 'reddit', 'puledotechupdate'}
{'reddit', '\x89û÷extremely', 'offensive\x89ûª'}
{'reddit', '\x89û÷extremely', 'offensive\x89ûª'}
{'\x89û', 'skanndtyagi', 'reddit'}
{'reddit'}
{'reddit'}
{'startups', 'reddit'}
{'reddit', '\x89û÷extremely', 'offensive\x89ûª'}
{'reddit', 'aannnnd'}
{'reddit'}
{'reddit', 'offensiveåêcontent'}
{'\x89û', 'reddit', 'hermancranston'}
{'reddit', 'huffman', 'cofounder', 'specif'}
{'reddit', 'huffman', 'cofounder', 'specif'}
{'reddit'}
{'reddit'}
{'onlinecommunities', 'reddit'}
{'reddit', 'huffman', 'cofounder'}
{'missambear'}
{'reddit'}
{'reddit', 'huffman', 'cofounder', 'specif'}
{'amageddon', 'reddit', 'offensi

In [ ]:
f = open('data_processed.csv', 'w')
df.to_csv(f)